# Разработка базы данных на примере софтверной компании BrunoyamSoft
Компания **BrunoyamSoft** занимается разработкой мобильных приложений с сфере аналитики данных, развлечений и бизнес-приложений. Компания широко представлена в маркетплейсах основных мобильных платформ и имеет большой портфель приложений различной тематики для удовлетворения требовательных пользователей.

Для контроля эффективности работы компании, была создана специальная база данных. База должна решать следующие задачи:
- Осуществлять контроль динамики продаж мобильных приложений и предоставлять аналитику по продажам. Хранить необходимые данные для этого;
- Осуществлять контроль над разработкой приложений. Связывать персонал с приложениями и их продажами.
- Предоставлять гибкие механизмы для конструирования произвольных запросов SQL в целях дальнейшего развития аналитических шаблонов.

### Разработаем схему хранения данных компании BrunoyamSoft.

База данных содержит следующие основные таблицы описывающие бизнес-логику компании
- **programmers** - таблица содержит список программистов, каждый из которых имеет имя, фамилию, а также ему присвоена квалификация;
- **languages** - таблица содержит список языков программирования, на которых разрабатываются приложения в компании;
- **qualification** - таблица содержит возможные квалификации, присваемые каждому программисту;
- **products** - таблица содержит разработанные программные продукты, их имя а также идентификатор OS для которой продукт разработан;
- **category** - содержит возможные значения категорий которые могут быть присвоены продукту;
- **os** - содержит возможные значения идентификаторов операционных систем (платформ). Каждый продукт может быть только для одной OS;
- **purshases** - таблица записей продаж. Содержит дату продажи и идентификатор продукта.
    
По мимо основных таблиц, существуют дополнительные связывающие таблицы, определяющие следующие требования:
- Каждый программист может знать несколько языков программирования. Один язык программирования может знать несколько программистов;
- Каждый программист может работать над несколькими продуктами. Один продукт как правило разрабатывает несколько программистов.

Для реализации данных требований, используется связь _```многие-ко-многим```_, для чего в базу данных добавлены следующие связывающие таблицы:
- **lang_knowledge** - таблица связывает программистов и языки программирования, которые они знают;
- **dev_products** - таблица связывает программистов и продукты которые они разрабатывают.

**Представим разработанную схему графически:**

![aa](./schema.png)

**Загрузим поддержку SQLite в Python и создадим указанные таблицы с помощью Pandas**

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path
import random
from datetime import datetime, date, time

Создадим функцию загрузки базы данных. Если база создается впервые, то выполняется скрипт ```init.sql``` по созданию таблиц и добавлению базовых записей в ней.

In [2]:
def get_sql():
    sql = sqlite3.connect('brunoyam.db')
    query = pd.read_sql('SELECT * FROM sqlite_master', sql)
    if query.name.count() > 0:
        return sql
    script = Path('init.sql').read_text()    
    sql.cursor().executescript(script)
    return sql

Создадим функцию ```init_purchares``` добавления записей продаж. Эти записи будут анализироваться в дальнейшем. Записи генерируются синтетические и только один раз - после создания таблицы. 

In [3]:
def gen_newpurshase(price_table):
    year = random.randint(2016, 2021)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    hour = random.randint(1, 23)
    m = random.randint(1, 59)
    d = datetime(year, month, day, hour, m)
    prod_id = random.randint(1, 11)
    price =  price_table['curr_price'][prod_id-1]
    return {'date' : d, 'product_id' : prod_id, 'price' : price}

def get_newpurchases(count, sql):
    products = pd.read_sql('SELECT * FROM products', sql)
    records = []   
    for x in range(count):
        record = gen_newpurshase(products)
        records.append(record)
    frame = pd.DataFrame(records)
    return frame

def init_purchares(sql):
    purchases = pd.read_sql('SELECT * FROM purchases', sql)
    if purchases.id.count() > 0:
        return
    records = get_newpurchases(1024, sql)
    nrecords = pd.DataFrame(records)
    nrecords.to_sql('purchases', sql, index=False, if_exists='append')
    sql.commit()
sql = get_sql()
init_purchares(sql)

Получим список продаж отсортированный по дате

In [4]:
purchases = pd.read_sql("SELECT date, name, price FROM 'purchases' JOIN 'products' ON products.id = product_id ORDER BY date", sql)
purchases 

,date,name,price
0,2016-01-01 20:09:00,Pazzle,0.00
1,2016-01-02 04:41:00,Mimo,21.30
2,2016-01-04 17:35:00,Sim Soti,0.00
3,2016-01-05 03:53:00,Sim Soti,0.00
4,2016-01-05 17:37:00,Tik Fok,12.98
...,...,...,...
1019,2021-12-19 21:30:00,Sim Soti,0.00
1020,2021-12-21 19:46:00,Tik Fok,12.98
1021,2021-12-25 11:53:00,Clev Calc,0.00
1022,2021-12-26 08:51:00,Tik Fok,12.98


Получим список программистов, сгрупированных по продуктам, которые они разрабатывают 

In [5]:
programmers_by_product = pd.read_sql("""
SELECT products.name, programmers.pname
FROM 'dev_products' 
JOIN 'products' ON products.id = product_id
JOIN 'programmers' ON programmers.id = programmers_id
""", sql)
programmers_by_product

,name,pname
0,Crossout Mobile,Steve Wozniak
1,Tik Fok,Guido van Rossum
2,Tik Fok,Steve Wozniak
3,Tik Fok,Bill Gates
4,Mimo,Steve Jobs
5,Mimo,Bill Gates
6,Mimo,Steve Jobs
7,Mimo,Bill Gates
8,EdX,Bjarne Stroustrup
9,Sim Soti,Bjarne Stroustrup


Посчитаем, какие программисты компании сколько языков программирования знают:

In [6]:
query = pd.read_sql("""
SELECT programmers.pname as 'Programmer', COUNT(language_id) as 'Lang. Knowing'
FROM 'lang_knowledge' 
JOIN 'languages' ON languages.id = language_id
JOIN 'programmers' ON programmers.id = programmers_id
GROUP BY Programmer
ORDER BY pname
""", sql)
query

,Programmer,Lang. Knowing
0,Bill Gates,1
1,Bjarne Stroustrup,1
2,Guido van Rossum,2
3,Steve Jobs,1
4,Steve Wozniak,2


Создадим обратный запрос: какой язык сколько программистов в компании знает

In [7]:
query = pd.read_sql("""
SELECT languages.name as 'Lang. Name', COUNT(programmers.pname) as 'Programmer'
FROM 'lang_knowledge' 
JOIN 'programmers' ON programmers.id = programmers_id
JOIN 'languages' ON languages.id = language_id
GROUP BY languages.name
""", sql)
query

,Lang. Name,Programmer
0,C#,2
1,C++,1
2,Java,2
3,Python,2


Получим список продуктов, над которыми работает хотя бы один сеньор

In [8]:
programmers_by_product = pd.read_sql("""
SELECT products.name as Product, programmers.pname as 'Programmer'
FROM 'dev_products' 
JOIN 'products' ON products.id = product_id
JOIN 'programmers' ON programmers.id = programmers_id
WHERE programmers.qualification_id = 3
ORDER BY pname
""", sql)
programmers_by_product

,Product,Programmer
0,EdX,Bjarne Stroustrup
1,Sim Soti,Bjarne Stroustrup
2,Clev Calc,Bjarne Stroustrup
3,Tik Fok,Guido van Rossum
4,Pazzle,Guido van Rossum
5,Funny Office,Guido van Rossum


Построим аналитику продаж. Расчитаем сумму продаж по каждому продукту. Бесплатные продукты с нулевым уровнем продаж исключим из аналитики

In [9]:
purchases = pd.read_sql("""
SELECT name as 'Product', SUM(price) as 'Total, $'
FROM 'purchases' 
JOIN 'products' ON products.id = product_id 
WHERE price > 0
GROUP BY name
ORDER BY price DESC
""", sql)
purchases 

,Product,"Total, $"
0,Mimo,2795.7
1,Tik Fok,1038.4
2,Funny Office,4630.5
3,Crossout Mobile,545.2
4,EdX,103.0


Сделаем аналогичный расчет, но по мобильным платформам. Рассчитаем сумму продаж для каждой мобильной платформы в отдельности

In [10]:
purchases = pd.read_sql("""
SELECT os.name as Platform, SUM(price) as 'Total, $'
FROM 'purchases' 
JOIN 'products' ON products.id = product_id 
JOIN 'os' ON os.id = products.os_id 
WHERE price > 0
GROUP BY os.name
ORDER BY price DESC
""", sql)
purchases 

,Platform,"Total, $"
0,iPhone,6703.3
1,Android,2306.5
2,SymphonyOS,103.0


Расчитаем количество продаж по категориям продуктов. Также добавим дополнительную колонку ```Qty```, показывающую сколько единиц товара было продано в каждой категории

In [11]:
purchases = pd.read_sql("""
SELECT category.name as Category, SUM(price) as 'Total, $', COUNT(category.name) as Qty
FROM 'purchases' 
JOIN 'products' ON products.id = product_id 
JOIN 'category' ON category.id = products.category_id 
WHERE price > 0
GROUP BY category.name
ORDER BY price DESC
""", sql)
purchases 

,Category,"Total, $",Qty
0,Game,1583.6,174
1,Office,4630.5,195
2,Education,2898.7,282
